In [1]:
import localgraphclustering as lgc

import time
import numpy as np

# Memory and time efficient loading of graphs. Memory and time grows linearly w.r.t the number of edges.

In [2]:
# Download this dataset from http://snap.stanford.edu/data/com-Orkut.html
# This dataset has 3,072,441 nodes and 117,185,083 edges.
# Loading this graph requires about 9GB of memory and 140 seconds on Macbook Pro 2016 with
# 2.9 GHz Intel Core i7, 16GB 2133 MHz LPDD3.
start = time.time()
g = lgc.graph_class_local('/Users/kimonfountoulakis/Downloads/com-orkut.ungraph.edgelist','edgelist','\t')
end = time.time()
print("running time:",str(end-start)+"s")

running time: 472.59764194488525s


# The routines below are applicable for scalable local graph clustering analytics; low memory requirements and fast running time. The memory and running time requirements are of the order of the volume of target cluster instead of the order of the volume of the graph.

# Call Approximate PageRank

In [3]:
start = time.time()

reference_node = 3215

# Find an approximate PageRank vector by calling a C++ code.
output_pr_fast = lgc.approximate_PageRank(g,[reference_node],cpp = True)

end = time.time()
print("running time:",str(end-start)+"s")

Uses the Andersen Chung and Lang (ACL) Algorithm.
running time: 0.7211651802062988s


# Round the output of Approximate PageRank

In [4]:
start = time.time()

# Call C++ version of sweep cut rounding on the l1-regularized PageRank solution.
output_sc_fast = lgc.sweep_cut(g,output_pr_fast,cpp = True)

end = time.time()
print("running time:",str(end-start)+"s")

running time: 1.3542461395263672s


# Memory efficient NCP

In [5]:
start = time.time()

ncp_instance = lgc.NCPData(g,do_largest_component=False)
df = ncp_instance.approxPageRank(ratio=0.5,timeout=7200)
ncp_plots = ncpplots.NCPPlots(df)
#plot conductance vs size
ncp_plots.cond_by_size()
#plot conductance vs volume
ncp_plots.cond_by_vol()
#plot isoperimetry vs size
ncp_plots.isop_by_size()

end = time.time()
print("running time:",str(end-start)+"s")

Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.


KeyboardInterrupt: 

Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.


# Compute the Network Community Profile of the largest component of the graph.

In [6]:
# The code below will compute the NCP for the largest connected component of the graph.
# To compute the largest component the memory requirements are being doubled, i.e., two times
# the size of the graph object in line 2 above.
start = time.time()

ncp_instance = lgc.NCPData(g,do_largest_component=True)
df = ncp_instance.approxPageRank(ratio=0.5,timeout=7200)
ncp_plots = ncpplots.NCPPlots(df)
#plot conductance vs size
ncp_plots.cond_by_size()
#plot conductance vs volume
ncp_plots.cond_by_vol()
#plot isoperimetry vs size
ncp_plots.isop_by_size()

end = time.time()
print("running time:",str(end-start)+"s")

Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.


KeyboardInterrupt: 

Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.


# Call L1-regularized PageRank

In [7]:
start = time.time()

reference_node = 3215

# Find an approximate PageRank vector by calling a C++ code.
output_l1reg_fast = lgc.approximate_PageRank(g,[reference_node],method = "l1reg",cpp = True)

end = time.time()
print("running time:",str(end-start)+"s")

Uses the Fast Iterative Soft Thresholding Algorithm (FISTA).
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
running time: 6.5954530239105225s


# Round the output of L1-regularizedPageRank

In [8]:
start = time.time()

# Call C++ version of sweep cut rounding on the l1-regularized PageRank solution.
output_sc_fast = lgc.sweep_cut(g,output_l1reg_fast,cpp = True)

end = time.time()
print("running time:",str(end-start)+"s")

Uses the Andersen Chung and Lang (ACL) Algorithm.
running time: 2.427741050720215s


# Memory efficient NCP based on L1-regularized PageRank

In [9]:
# The code below will compute the NCP of graph and not the NCP of the largest connected component.
# The code below does not double the memory requirements.

start = time.time()

ncp_instance = lgc.NCPData(g,do_largest_component=False)
df = ncp_instance.l1reg(ratio=0.5,timeout=7200)
ncp_plots = ncpplots.NCPPlots(df)
#plot conductance vs size
ncp_plots.cond_by_size()
#plot conductance vs volume
ncp_plots.cond_by_vol()
#plot isoperimetry vs size
ncp_plots.isop_by_size()
end = time.time()
print("running time:",str(end-start)+"s")

end = time.time()
print("running time:",str(end-start)+"s")

Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Fast Iterative Soft Thresholding Algorithm (FISTA).
Uses the Andersen Chung and Lang (ACL) Algorithm.
Uses the Fast Iterative Soft Thresholding Algorithm (FISTA).
Uses the Fast Iterative Soft Thresholding Algorithm (FISTA).
Uses the Fast Iterative Soft Thresholding Algorithm (FISTA).
Uses the Andersen Chung and Lang (ACL) Algorithm.


KeyboardInterrupt: 

# More to come. We will have scalable implementations for l1-regularized PageRank and Capacity Releasing Diffusion soon.